<a href="https://colab.research.google.com/github/Sonacdac14/768_image_unet/blob/master/Course_2_Mod_2_Topic_list_and_Reading_list_(2)_docx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir sonali

In [ ]:
!cat /content/sonali/Course 2 Mod 2 Topic list and Reading list (2).docx

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `cat /content/sonali/Course 2 Mod 2 Topic list and Reading list (2).docx'


In [ ]:
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 7.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-a7rz71wi
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-a7rz71wi
  Resolved https://github.com/deepset-ai/haystack.git to commit 255072d8d548a19a1678ddc46b41d41cf5d09bc4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader

In [ ]:
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

In [ ]:
from haystack.document_stores import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

In [ ]:
doc_dir = "/content/sonali"
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry
INFO:haystack.utils.preprocessing:Converting /content/sonali/Course 2 Mod 2 Topic list and Reading list (2).docx


In [ ]:
print(docs[:3])
document_store.write_documents(docs)

[<Document: {'content': 'Course 2 Module 2: Organizational ethics\nMLO1: Articulate the purpose and benefits of applying an ethical framework within medical practice \nMLO2: Critically evaluate the challenges of balancing personal and professional ethics in the context of a commercial environment. \nTopic 1 - Ethical issues in healthcare compliance - Balancing Commerce with the Healthcare Calling \nTopic 2 - Designing an ethical organization - Balancing Personal and Professional ethics in healthcare\nTopic 3 - Blockchain Ethical Design \nTopic 4 - Difference between Organizational and Healthcare ethics and the Connection to Patient Trust \nTopic 5 - Ethical considerations in end-of-life discussion - quality vs $$\t\t\t\nTopic 6 - How much do ethics play a part in healthcare regulatory development?\nTopic 7 - The Roles of the Ethics and Peer Review Committees in Organizational Ethics\nTopic 8 - Balancing personal and professional ethics in healthcare\nTopic 9 - Ethical relationships bet

In [ ]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [ ]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
prediction = pipe.run(
    query="Blockchain Ethical Design ?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples: 100%|██████████| 1/1 [00:02<00:00,  2.45s/ Batches]


In [ ]:
from pprint import pprint

pprint(prediction['answers'])

[<Answer {'answer': 'Topic 3', 'type': 'extractive', 'score': 0.23008516430854797, 'context': 'organization - Balancing Personal and Professional ethics in healthcare\nTopic 3 - Blockchain Ethical Design \nTopic 4 - Difference between Organization', 'offsets_in_document': [{'start': 476, 'end': 483}], 'offsets_in_context': [{'start': 72, 'end': 79}], 'document_id': 'da0d1c38238d28a96b4bf89b401e3593', 'meta': {'name': 'Course 2 Mod 2 Topic list and Reading list (2).docx'}}>,
 <Answer {'answer': 'Blockchain ethical design framework for healthcare', 'type': 'extractive', 'score': 0.05042167380452156, 'context': 'ECESSARY? - Lech Keller-Krawczyk \n• PDF Available Blockchain ethical design framework for healthcare – The GBA Healthcare Working Group - Marquis Alle', 'offsets_in_document': [{'start': 1425, 'end': 1475}], 'offsets_in_context': [{'start': 50, 'end': 100}], 'document_id': 'da0d1c38238d28a96b4bf89b401e3593', 'meta': {'name': 'Course 2 Mod 2 Topic list and Reading list (2).docx'}}

In [ ]:
from haystack.pipelines import DocumentSearchPipeline
pipeline = DocumentSearchPipeline(retriever)
query = ' Ethical issues in healthcare '
result = pipeline.run(query, params={"Retriever": {"top_k": 10}})

In [ ]:
from haystack.utils import print_documents

print_documents(result, max_text_len=100, print_name=True, print_meta=True)


Query:  Ethical issues in healthcare 

{   'content': 'Course 2 Module 2: Organizational ethics\n'
               'MLO1: Articulate the purpose and benefits of applying an et...',
    'meta': {'name': 'Course 2 Mod 2 Topic list and Reading list (2).docx'},
    'name': 'Course 2 Mod 2 Topic list and Reading list (2).docx'}



In [ ]:
[x.to_dict() for x in result["documents"]]

[{'content': 'Course 2 Module 2: Organizational ethics\nMLO1: Articulate the purpose and benefits of applying an ethical framework within medical practice \nMLO2: Critically evaluate the challenges of balancing personal and professional ethics in the context of a commercial environment. \nTopic 1 - Ethical issues in healthcare compliance - Balancing Commerce with the Healthcare Calling \nTopic 2 - Designing an ethical organization - Balancing Personal and Professional ethics in healthcare\nTopic 3 - Blockchain Ethical Design \nTopic 4 - Difference between Organizational and Healthcare ethics and the Connection to Patient Trust \nTopic 5 - Ethical considerations in end-of-life discussion - quality vs $$\t\t\t\nTopic 6 - How much do ethics play a part in healthcare regulatory development?\nTopic 7 - The Roles of the Ethics and Peer Review Committees in Organizational Ethics\nTopic 8 - Balancing personal and professional ethics in healthcare\nTopic 9 - Ethical relationships between the he